In [10]:
%run functions.ipynb

In [11]:
from google.cloud import storage
import pandas as pd
from io import BytesIO

# Your GCS bucket name
bucket_name = 'ccai-storage'

# Paths to the Parquet files in your GCS bucket
articles_parquet_path = 'pipeline/final_df3.parquet'

# Initialize the GCS client
client = storage.Client()

# Load the Parquet files
final_df3 = load_parquet_from_gcs(bucket_name, articles_parquet_path)

In [12]:
print(final_df3['split'].value_counts())

split
TRAIN    100
Name: count, dtype: int64


In [13]:
final_df3

,uri,url,title,body,date,time,dateTime,dateTimePub,lang,isDuplicate,...,Violent,Firearms__Weapons,Public_Safety,Health,Religion__Belief,Illicit_Drugs,War__Conflict,Politics,Finance,Legal
0,7582830489,https://www.ilfattoquotidiano.it/2023/06/07/pr...,"Premio Strega 2023, quattro scrittrici in fina...",Quattro scrittrici in cinquina alla finale del...,2023-06-07,19:32:00,2023-06-07T19:32:00Z,2023-06-07T19:29:00Z,ita,False,...,0.002514,0.000000,0.005008,0.004200,0.002467,0.002336,0.000477,0.004959,0.007634,0.002938
1,7358061043,https://www.corriere.it/spettacoli/23_gennaio_...,La rivista Usa The Atlantic stronca i Maneskin...,Il giornalista Spencer Kornhaber fa a pezzi Da...,2023-01-19,18:21:00,2023-01-19T18:21:00Z,2023-01-19T18:10:00Z,ita,False,...,0.028125,0.000000,0.010695,0.004200,0.001459,0.002336,0.010417,0.093664,0.003163,0.011976
2,7376407793,https://www.tgcom24.mediaset.it/people/damiano...,"Damiano David, shopping in gioielleria con Gio...",Per Giorgia Soleri (e Damiano) la gravidanza p...,2023-01-31,09:44:00,2023-01-31T09:44:00Z,2023-01-31T09:43:00Z,ita,False,...,0.002584,0.000000,0.005282,0.626168,0.019215,0.057377,0.010417,0.030905,0.006837,0.011976
3,7807122008,https://www.ilfattoquotidiano.it/2023/10/29/x-...,"X Factor, Francesca Michielin e Ambra Angiolin...","Ieri sera, sabato 28 ottobre, Francesca Michie...",2023-10-29,11:03:39,2023-10-29T11:03:39Z,2023-10-29T11:00:59Z,ita,False,...,0.000000,0.000000,0.001439,0.023965,0.002467,0.009554,0.000477,0.002819,0.001149,0.002938
4,7539580387,https://www.corriere.it/spettacoli/23_maggio_1...,Alessandro Cecchi Paone e Simone Antolini hann...,Ancora sconosciute le motivazioni della scelta...,2023-05-11,15:43:00,2023-05-11T15:43:00Z,2023-05-11T15:34:00Z,ita,False,...,0.014493,0.000000,0.005008,0.005747,0.012508,0.009554,0.000477,0.059867,0.003163,0.011976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7566215221,https://www.ilfattoquotidiano.it/2023/05/28/kl...,"Kledi Kadiu: ""Non sono morto... Scrivono che m...","""Si fa tutto per avere un click in più, questa...",2023-05-28,10:28:00,2023-05-28T10:28:00Z,2023-05-28T10:22:00Z,ita,False,...,0.641459,0.076923,0.410853,0.213836,0.053885,0.294118,0.218391,0.391892,0.007634,0.176871
96,7534884888,https://www.avvenire.it/opinioni/pagine/patric...,La tragedia di Foggia. Jessica uccisa dal padr...,"Più sono giovani, i figli, più vanno seguiti, ...",2023-05-08,20:59:00,2023-05-08T20:59:00Z,2023-05-08T19:14:00Z,ita,False,...,0.793103,0.018182,0.256757,0.050201,0.352273,0.057377,0.066038,0.030905,0.007634,0.100946
97,7826647045,https://www.ilfattoquotidiano.it/2023/11/09/ro...,Robot spinge un uomo contro il nastro trasport...,Per l'uomo a nulla è servito il disperato tent...,2023-11-09,13:22:48,2023-11-09T13:22:48Z,2023-11-09T13:20:56Z,ita,False,...,0.773973,0.076923,0.594771,0.950521,0.012508,0.009554,0.141892,0.011268,0.041322,0.546099
98,7611687004,https://www.ilfattoquotidiano.it/2023/06/26/lu...,"'Lucio Battisti, il genio invisibile', il libr...",Dopo l'infornata di riconoscimenti ricevuti pe...,2023-06-26,08:27:00,2023-06-26T08:27:00Z,2023-06-26T08:24:00Z,ita,False,...,0.002514,0.000000,0.001439,0.004200,0.012508,0.009554,0.006250,0.008380,0.003163,0.002938


In [14]:
from google.cloud import language
import pandas as pd

def analyze_text_entities(text: str) -> language.AnalyzeEntitiesResponse:
    client = language.LanguageServiceClient()
    document = language.Document(
        content=text,
        type_=language.Document.Type.PLAIN_TEXT,
    )
    return client.analyze_entities(document=document)

In [19]:
def analyze_document_entities(df: pd.DataFrame, num_documents: int = None, column:str = 'text'):
    if num_documents is None:
        num_documents = len(df)
    
    documents = []

    for index, row in df.iterrows():
        if len(documents) >= num_documents:
            break  # Stop after processing the specified number of documents

        linetext_cleaned = row.get(column, '')

        if linetext_cleaned:
            # Perform entity analysis on the cleaned text
            response = analyze_text_entities(linetext_cleaned)
            entity_type_counts = {}
            entities_data = {}

            for entity in response.entities:
                entity_type = entity.type_.name
                salience = entity.salience

                # Update entity counts
                entity_type_counts[entity_type] = entity_type_counts.get(entity_type, 0) + 1

                # Append salience to the list corresponding to the entity type
                if entity_type not in entities_data:
                    entities_data[entity_type] = []
                    #print(entities_data)
                entities_data[entity_type].append(salience)

            # Calculate mean salience for each entity type
            mean_salience_per_entity_type = {}
            for entity_type, salience_list in entities_data.items():
                mean_salience_per_entity_type[entity_type] = sum(salience_list) / len(salience_list)
                #print(mean_salience_per_entity_type)

            # Create a copy of the current row as a dict and update it with entity counts
            document = row.to_dict()
            document.update(entity_type_counts)

            # Add mean salience for each entity type to the document
            for entity_type, mean_salience in mean_salience_per_entity_type.items():
                document[f"{entity_type}_mean_salience"] = mean_salience

            documents.append(document)




    # Create a new DataFrame that includes the original data and the new entity count columns
    df_result = pd.DataFrame(documents)
    
    # Fill missing entity count values with 0
    #entity_columns = [col for col in df_result.columns if col not in df.columns]
    #df_result[entity_columns] = df_result[entity_columns].fillna(0).astype(int)

    return df_result


In [20]:
import time

start_time = time.time()
df_entities = analyze_document_entities(final_df3, column="body_pre")
end_time = time.time()

execution_time = end_time - start_time
execution_time

58.35318207740784

In [21]:
nan_count = df_entities.isna().sum()
print("Total NaN values:", nan_count)

Total NaN values: uri                       0
url                       0
title                     0
body                      0
date                      0
                         ..
DATE_mean_salience       49
PRICE                    96
PRICE_mean_salience      96
ADDRESS                  99
ADDRESS_mean_salience    99
Length: 73, dtype: int64


In [26]:
df_copy=df_entities.copy()

In [28]:
# Fill missing entity count values with 0
entity_columns = [col for col in df_copy.columns if col not in final_df3.columns]
df_copy[entity_columns] = df_copy[entity_columns].fillna(0).astype(float)
# Replace null values with zero in place

In [29]:
nan_count = df_copy.isna().sum()
print("Total NaN values:", nan_count)

Total NaN values: uri                            0
title                          0
body                           0
date                           0
dateTime                       0
authors                        0
sharesFacebook                 0
categoryLabels                 0
sourceTitle                    0
topic                          0
split                          0
translated_text                0
prepr_title                    0
title_eng                      0
title_eng_pre                  0
preprocessed_text              0
shares_scaled                  0
score                          0
magnitude                      0
Toxic                          0
Insult                         0
Profanity                      0
Derogatory                     0
Sexual                         0
Death_Harm__Tragedy            0
Violent                        0
Firearms__Weapons              0
Public_Safety                  0
Health                         0
Religion__Belief         

In [30]:
df_copy.describe()

,date,sharesFacebook,shares_scaled,score,magnitude,Toxic,Insult,Profanity,Derogatory,Sexual,...,ORGANIZATION_mean_salience,EVENT_mean_salience,PERSON_mean_salience,OTHER_mean_salience,PRICE_mean_salience,NUMBER_mean_salience,WORK_OF_ART,CONSUMER_GOOD,WORK_OF_ART_mean_salience,CONSUMER_GOOD_mean_salience
count,3614,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,...,3614.000000,3614.000000,3614.000000,3614.000000,3614.0,3614.0,3614.000000,3614.000000,3614.000000,3614.000000
mean,2023-11-15 02:25:26.065301760,26.220808,0.842534,-0.221721,0.552324,0.101783,0.087084,0.038014,0.029261,0.008280,...,0.019805,0.017483,0.025626,0.021075,0.0,0.0,1.925291,1.053680,0.012363,0.006258
min,2023-10-17 00:00:00,0.000000,-0.250000,-0.800000,0.000000,0.014622,0.010394,0.002904,0.003574,0.002435,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
25%,2023-11-09 00:00:00,1.000000,-0.208333,-0.500000,0.200000,0.052281,0.043235,0.015939,0.008934,0.002435,...,0.006941,0.004150,0.011081,0.009424,0.0,0.0,0.000000,0.000000,0.000000,0.000000
50%,2023-11-14 00:00:00,6.000000,0.000000,-0.300000,0.500000,0.077808,0.070100,0.023985,0.018461,0.002435,...,0.012709,0.010810,0.018902,0.015111,0.0,0.0,1.000000,0.000000,0.005550,0.000000
75%,2023-11-18 00:00:00,25.000000,0.791667,0.000000,0.700000,0.136032,0.114481,0.050467,0.036435,0.002470,...,0.024075,0.021994,0.032618,0.026655,0.0,0.0,3.000000,1.000000,0.015692,0.007128
max,2023-12-21 00:00:00,1802.000000,74.833333,0.900000,4.300000,0.533487,0.548359,0.585504,0.468937,0.628449,...,0.373596,0.234057,0.337606,0.526679,0.0,0.0,32.000000,68.000000,0.410668,0.217651
std,NaN,75.916881,3.163203,0.342070,0.445302,0.071930,0.067254,0.039583,0.036336,0.044657,...,0.024283,0.023386,0.023729,0.019777,0.0,0.0,2.815767,2.440637,0.021755,0.014577


In [11]:
df_copy.columns

Index(['uri', 'title', 'body', 'date', 'dateTime', 'authors', 'sharesFacebook',
       'categoryLabels', 'sourceTitle', 'topic', 'split', 'translated_text',
       'prepr_title', 'title_eng', 'title_eng_pre', 'preprocessed_text',
       'shares_scaled', 'score', 'magnitude', 'Toxic', 'Insult', 'Profanity',
       'Derogatory', 'Sexual', 'Death_Harm__Tragedy', 'Violent',
       'Firearms__Weapons', 'Public_Safety', 'Health', 'Religion__Belief',
       'Illicit_Drugs', 'War__Conflict', 'Politics', 'Finance', 'Legal',
       'LOCATION', 'ORGANIZATION', 'EVENT', 'PERSON', 'OTHER', 'PRICE',
       'NUMBER', 'LOCATION_mean_salience', 'ORGANIZATION_mean_salience',
       'EVENT_mean_salience', 'PERSON_mean_salience', 'OTHER_mean_salience',
       'PRICE_mean_salience', 'NUMBER_mean_salience', 'WORK_OF_ART',
       'CONSUMER_GOOD', 'WORK_OF_ART_mean_salience',
       'CONSUMER_GOOD_mean_salience'],
      dtype='object')

In [31]:
df_copy.describe()

,date,sharesFacebook,shares_scaled,score,magnitude,Toxic,Insult,Profanity,Derogatory,Sexual,...,ORGANIZATION_mean_salience,EVENT_mean_salience,PERSON_mean_salience,OTHER_mean_salience,PRICE_mean_salience,NUMBER_mean_salience,WORK_OF_ART,CONSUMER_GOOD,WORK_OF_ART_mean_salience,CONSUMER_GOOD_mean_salience
count,3614,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,...,3614.000000,3614.000000,3614.000000,3614.000000,3614.0,3614.0,3614.000000,3614.000000,3614.000000,3614.000000
mean,2023-11-15 02:25:26.065301760,26.220808,0.842534,-0.221721,0.552324,0.101783,0.087084,0.038014,0.029261,0.008280,...,0.019805,0.017483,0.025626,0.021075,0.0,0.0,1.925291,1.053680,0.012363,0.006258
min,2023-10-17 00:00:00,0.000000,-0.250000,-0.800000,0.000000,0.014622,0.010394,0.002904,0.003574,0.002435,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
25%,2023-11-09 00:00:00,1.000000,-0.208333,-0.500000,0.200000,0.052281,0.043235,0.015939,0.008934,0.002435,...,0.006941,0.004150,0.011081,0.009424,0.0,0.0,0.000000,0.000000,0.000000,0.000000
50%,2023-11-14 00:00:00,6.000000,0.000000,-0.300000,0.500000,0.077808,0.070100,0.023985,0.018461,0.002435,...,0.012709,0.010810,0.018902,0.015111,0.0,0.0,1.000000,0.000000,0.005550,0.000000
75%,2023-11-18 00:00:00,25.000000,0.791667,0.000000,0.700000,0.136032,0.114481,0.050467,0.036435,0.002470,...,0.024075,0.021994,0.032618,0.026655,0.0,0.0,3.000000,1.000000,0.015692,0.007128
max,2023-12-21 00:00:00,1802.000000,74.833333,0.900000,4.300000,0.533487,0.548359,0.585504,0.468937,0.628449,...,0.373596,0.234057,0.337606,0.526679,0.0,0.0,32.000000,68.000000,0.410668,0.217651
std,NaN,75.916881,3.163203,0.342070,0.445302,0.071930,0.067254,0.039583,0.036336,0.044657,...,0.024283,0.023386,0.023729,0.019777,0.0,0.0,2.815767,2.440637,0.021755,0.014577


In [33]:
print(df_copy['split'].value_counts())

split
VALIDATION    1303
TEST          1303
TRAIN         1008
Name: count, dtype: int64


In [ ]:
### Salvo il DF to the storage

In [32]:
### Salvo il df come parquet to the bucket
bucket_name="ccai-storage"

folder = "pipeline"

# Define your Google Cloud Storage bucket name and Parquet file name
parquet_file_name = 'final_df4.parquet'

# Save DataFrames to GCS as Parquet files in the 'fabio' folder
save_df_to_gcs_parquet(df_copy, bucket_name, folder, parquet_file_name)